In [134]:
from pymongo import MongoClient
from flask import Flask, request, jsonify
import json
import requests
import pandas as pd
import csv

In [135]:
mongo = MongoClient(port=27017)
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'establishments', 'fruits_db', 'fruitsdb', 'local', 'met', 'petsitly_marketing', 'player2_weather', 'travel_db', 'uk_food', 'weather_database']


In [ ]:
# Define  filepath containing Stormglass API Key
file_path = './keys/stormglass.txt'

# Read the API key from the file
with open(file_path, 'r') as file:
    stormglass_api_key = file.read().strip()

In [ ]:
mongo = MongoClient('mongodb://localhost:27017')
db = mongo['player2_weather']
charleston_air = db['charleston_air']
endpoint = 'https://api.stormglass.io/v2/weather/point'
params = {
            'lat': 26.443398,
            'lng': -82.111511,
            'params': 'airTemperature',
            'start': '2023-09-23',
            'end': '2023-09-30'
            
        }
headers = {
        'Authorization': stormglass_api_key
    }
response = requests.get(endpoint, params=params, headers=headers)
response.raise_for_status()
data = response.json()
sanibel_air.insert_one(data)
if sanibel_air.count_documents({}) > 0:
    print("Data inserted into sanibel_data collection.")
else:
    print("Failed to insert data.")


In [ ]:
data = sanibel_air.find()
for document in data:
    pprint(document)

In [ ]:
data = list(sanibel_air.find())
pprint(data)
json_data = json.dumps(data, default=str)
with open('Resources/sanibel_air.json', 'w') as json_file:
    json_file.write(json_data)

In [ ]:
mongo = MongoClient('mongodb://localhost:27017')
db = mongo['player2_weather']
charleston_air = db['charleston_air']
endpoint = 'https://api.stormglass.io/v2/weather/point'
params = {
            'lat': 32.7765,
            'lng': -79.93092,
            'params': 'airTemperature',
            'start': '2023-09-23',
            'end': '2023-09-30'
            
        }
headers = {
        'Authorization': stormglass_api_key
    }
response = requests.get(endpoint, params=params, headers=headers)
response.raise_for_status()
data = response.json()
charleston_air.insert_one(data)
if charleston_air.count_documents({}) > 0:
    print("Data inserted into charleston collection.")
else:
    print("Failed to insert data.")

In [ ]:
data = charleston_air.find()
for document in data:
    pprint(document)

In [ ]:
data = list(charleston_air.find())
pprint(data)
json_data = json.dumps(data, default=str)
with open('Resources/charleston_air.json', 'w') as json_file:
    json_file.write(json_data)

In [ ]:
data = charleston_air.find()
for document in data:
    pprint(document) 

In [ ]:
data = charleston_air.find_one({}, {"_id": 0, "hours": 1})['hours']
csv_file = 'Resources/chairtemps.csv'
fields = ['time', 'airTemperature.noaa', 'airTemperature.sg']
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(fields)
    
    for entry in data:
        row_data = [
            entry['time'],
            entry['airTemperature']['noaa'],
            entry['airTemperature']['sg']
        ]
        writer.writerow(row_data)

In [ ]:
data = sanibel_air.find_one({}, {"_id": 0, "hours": 1})['hours']
csv_file = 'Resources/sanibelairtemps.csv'
fields = ['time', 'airTemperature.noaa', 'airTemperature.sg']
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(fields)
    
    for entry in data:
        row_data = [
            entry['time'],
            entry['airTemperature']['noaa'],
            entry['airTemperature']['sg']
        ]
        writer.writerow(row_data)

In [138]:
data = charleston_air.find({[1]}, {"_id": 0, "hours": 1})['hours']
csv_file = 'Resources/chairtemps.csv'
fields = ['time', 'airTemperature.noaa', 'airTemperature.sg']
with open(csv_file, 'a', newline = '') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(fields)
    
        for entry in item['hours']:
            row_data = [
                item['_id'],
                entry['time'],
                entry['airTemperature']['noaa'],
                entry['airTemperature']['sg']
            ]
            writer.writerow(row_data)

KeyError: 'hours'

In [156]:
with open('Resources/sanibel_air.json') as file:
    data = json.load(file)

# Access the second dataset by indexing the data list using [1]
second_dataset = data[1]
third_dataset = data[2]

# Specify the CSV file path
csv_file = 'Resources/sanibelairtemps.csv'

# Define the header fields for your CSV file
fields = ['time', 'airTemperature.noaa', 'airTemperature.sg']

# Write the data from the second dataset to the CSV file
with open(csv_file, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header fields
    writer.writerow(fields)
    
    # Iterate over the data in the "hours" field of the second dataset
    for entry in third_dataset['hours']:
        # Extract the required data from each entry
        row_data = [
            entry['time'],
            entry['airTemperature']['noaa'],
            entry['airTemperature']['sg']
        ]
        
        # Write the row_data to the CSV file
        writer.writerow(row_data)